In [1]:
import grequests, json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from requests.auth import HTTPBasicAuth
from datetime import datetime
from time import sleep
import pytz
from pyModbusTCP.client import ModbusClient
from IPython.display import clear_output
import signal
from contextlib import contextmanager

PI_string = ['https://piwebapi.caserones.cl:8443/piwebapi/streamsets/F1EmiGOb_efVU0GUF6kOZIrFCwjlZ8dUmz6xGCEwBQVpZaMgTUxDQ1BJQUYwMVxQSVdFQlNFUlZJQ0VTXEJPTUJBU1w5NDAwVlw5NDAwLVBQLTAxMQ/value',
        'https://piwebapi.caserones.cl:8443/piwebapi/streamsets/F1EmiGOb_efVU0GUF6kOZIrFCw1hLobEmz6xGCEwBQVpZaMgTUxDQ1BJQUYwMVxQSVdFQlNFUlZJQ0VTXEJPTUJBU1w5NDAwVlw5NDAwLVBQLTAxMg/value',
        'https://piwebapi.caserones.cl:8443/piwebapi/streamsets/F1EmiGOb_efVU0GUF6kOZIrFCwvBQvXkmz6xGCEwBQVpZaMgTUxDQ1BJQUYwMVxQSVdFQlNFUlZJQ0VTXEJPTUJBU1w5NDAwVlw5NDAwLVBQLTAxMw/value',
        'https://piwebapi.caserones.cl:8443/piwebapi/streamsets/F1EmiGOb_efVU0GUF6kOZIrFCwynLRVUmz6xGCEwBQVpZaMgTUxDQ1BJQUYwMVxQSVdFQlNFUlZJQ0VTXEJPTUJBU1w5NDAwVlw5NDAwLVBQLTAxNA/value',
        'https://piwebapi.caserones.cl:8443/piwebapi/streamsets/F1EmiGOb_efVU0GUF6kOZIrFCw3l5GQEmz6xGCEwBQVpZaMgTUxDQ1BJQUYwMVxQSVdFQlNFUlZJQ0VTXEJPTUJBU1w5NDAwVlw5NDAwLVBQLTAxNQ/value',
        'https://piwebapi.caserones.cl:8443/piwebapi/streamsets/F1EmiGOb_efVU0GUF6kOZIrFCwQF3BlUiz6xGCEwBQVpZaMgTUxDQ1BJQUYwMVxQSVdFQlNFUlZJQ0VTXEJPTUJBU1w5NDAwVlw5NDAwLVBQLTAxNg/value',]

now = datetime.now()
new_timezone = pytz.timezone("Etc/GMT+4")
client_par = ModbusClient(host="192.168.168.105", port=1234)
client_par.open()

True

In [2]:
nombre_archivo = 'PAR_{}-{}-{}.xlsx'.format(now.year,now.month,now.day)
par_pp11, par_pp12, par_pp13 = pd.DataFrame(columns = []), pd.DataFrame(columns = []), pd.DataFrame(columns = [])
par_pp14, par_pp15, par_pp16 = pd.DataFrame(columns = []), pd.DataFrame(columns = []), pd.DataFrame(columns = [])
with pd.ExcelWriter('~/Escritorio/Caserones/datos/PAR/'+nombre_archivo) as writer:  
    par_pp11.to_excel(writer,sheet_name='PP-011')
    par_pp12.to_excel(writer,sheet_name='PP-012')
    par_pp13.to_excel(writer,sheet_name='PP-013')
    par_pp14.to_excel(writer,sheet_name='PP-014')
    par_pp15.to_excel(writer,sheet_name='PP-015')
    par_pp16.to_excel(writer,sheet_name='PP-016')

In [3]:
@contextmanager
def timeout(time):
    # Register a function to raise a TimeoutError on the signal.
    signal.signal(signal.SIGALRM, raise_timeout)
    # Schedule the signal to be sent after ``time``.
    signal.alarm(time)

    try:
        yield
    except TimeoutError:
        pass
    finally:
        # Unregister the signal so it won't be triggered
        # if the timeout is not reached.
        signal.signal(signal.SIGALRM, signal.SIG_IGN)

def raise_timeout(signum, frame):
    raise TimeoutError
    
def pedir_datos(pi_string):
    with timeout(30):
        reqs = [grequests.get(pi_string,auth=HTTPBasicAuth('vulco.esed','vpump$mlcc2021'))]
        valores = grequests.map(reqs)
    return valores

In [ ]:
while True:
    try:
        now = datetime.now()
        if (nombre_archivo != 'PAR_{}-{}-{}.xlsx'.format(now.year,now.month,now.day)):
            par_pp11, par_pp12, par_pp13 = pd.DataFrame(columns = []), pd.DataFrame(columns = []), pd.DataFrame(columns = [])
            par_pp14, par_pp15, par_pp16 = pd.DataFrame(columns = []), pd.DataFrame(columns = []), pd.DataFrame(columns = [])

        clear_output(wait=True)
        
        print("PAR")
        ###
        print('PP-011')
        print("Solicitando datos")
        try:
            valores = pedir_datos(PI_string[0])
            pi_av = 1
            time_stamp = json.loads(valores[0].text)['Items'][0]['Value']['Timestamp']
            corr_total = json.loads(valores[0].text)['Items'][0]['Value']['Value']
            hor_total = json.loads(valores[0].text)['Items'][1]['Value']['Value']
            pot_act = json.loads(valores[0].text)['Items'][2]['Value']['Value']
            temp_descA = json.loads(valores[0].text)['Items'][3]['Value']['Value'] # Temperatura descanso ventilacion
            temp_descB = json.loads(valores[0].text)['Items'][4]['Value']['Value'] # Temperatura carga
            temp_dev1A = json.loads(valores[0].text)['Items'][5]['Value']['Value'] # Temperatura devanado 1
            temp_dev1B = json.loads(valores[0].text)['Items'][6]['Value']['Value'] # Temperatura devanado 2
            vdf = json.loads(valores[0].text)['Items'][7]['Value']['Value']
            velocidad_bomba = json.loads(valores[0].text)['Items'][8]['Value']['Value']
            hor_op = json.loads(valores[0].text)['Items'][9]['Value']['Value']
            pump_run = json.loads(valores[0].text)['Items'][10]['Value']['Value']
    
            time_stamp = ((pd.to_datetime(time_stamp,format='%Y-%m-%d %H:%M:%S')).astimezone(new_timezone)).strftime("%Y-%m-%d %H:%M:%S")
            time_stamp_epoch = int((datetime.strptime(time_stamp, "%Y-%m-%d %H:%M:%S")).timestamp())
            tsepoch_bin = bin(time_stamp_epoch)[2:]
            while (len(tsepoch_bin) < 32):
                tsepoch_bin = "0"+ tsepoch_bin
            tsepoch_int_high = int(tsepoch_bin[0:16],2)
            tsepoch_int_low = int(tsepoch_bin[16:],2)
            
            hor_total_bin = bin(np.int64(hor_total))[2:]
            while (len(hor_total_bin) < 48):
                hor_total_bin = "0"+ hor_total_bin
            hor_total_hhigh = int(hor_total_bin[0:16],2)
            hor_total_high = int(hor_total_bin[16:32],2)
            hor_total_low = int(hor_total_bin[32:48],2)

            hor_logica, estado, ef_inst, velocidad_motor, corr_A, corr_B, corr_C  = 1, 0, 0, 0, 0, 0, 0
            temp_dev1C, temp_dev2A, temp_dev2B, temp_dev2C, pres_des, flow, temp_bomba = 0, 0, 0, 0, 0, 0, 0
            acel_hor, acel_ver, acel_ax, vel_hor, vel_ver, vel_ax, p2p_x, p2p_y, p2p_z =  0, 0, 0, 0, 0, 0, 0, 0, 0
            id_sensor, bateria_sensor, rssi_sensor = 0, 0, 0
            
            pump_health = 93
            motor_health = 93
            eff_ind = 91
            operating_hours = np.int16(hor_op)
            ovall_he_pef = np.int16(np.amin([pump_health,motor_health,eff_ind]))
    
            print("Escribiendo al Modbus")
            client_par.write_multiple_registers(250,[np.int16(pump_run), hor_logica, hor_total_hhigh, hor_total_high, hor_total_low, np.int16(hor_op),
                                                      tsepoch_int_high, tsepoch_int_low, np.int16(estado), np.int16(ef_inst*100)])
            client_par.write_multiple_registers(260,[np.int16(velocidad_motor),np.int16(corr_total), np.int16(corr_A), np.int16(corr_B), np.int16(corr_C),
                                                      np.int16(pot_act),np.int16(temp_dev1A),np.int16(temp_dev1B),np.int16(temp_dev1C),np.int16(temp_dev2A)])
            client_par.write_multiple_registers(270,[np.int16(temp_dev2B),np.int16(temp_dev2C),np.int16(temp_descA),np.int16(temp_descB), np.int16(pres_des),
                                                      np.int16(flow),np.int16(velocidad_bomba),np.int16(temp_bomba),np.int16(acel_hor),np.int16(acel_ver)])
            client_par.write_multiple_registers(280,[np.int16(acel_ax),np.int16(vel_hor),np.int16(vel_ver),np.int16(vel_ax), np.int16(p2p_x), np.int16(p2p_y),
                                                      np.int16(p2p_z),np.int16(id_sensor),np.int16(bateria_sensor),np.int16(rssi_sensor)])
            client_par.write_multiple_registers(290,[pi_av, pump_health, motor_health, eff_ind, operating_hours, ovall_he_pef])        
        
        
            print("Guardando dataframe")
            pp11 = pd.DataFrame({'PI Disponible': [pi_av],'Pump Running':[pump_run],'Horometro Lógica':[hor_logica],'Horometro total':[hor_total],'Horas operando':[hor_op],
                                 'TimeStamp':[time_stamp],'Estado bomba':[estado],'Eficiencia instantanea':[ef_inst],'Velocidad motor':[velocidad_motor],
                                 'Corriente total':[corr_total],'Corriente A':[corr_A],'Corriente B':[corr_B],'Corriente C':[corr_C],
                                 'Potencia Activa':[pot_act],'Temperatura devanado 1A':[temp_dev1A],'Temperatura devanado 1B':[temp_dev1B],
                                 'Temperatura devanado 1C':[temp_dev1C],'Temperatura devanado 2A':[temp_dev2A],'Temperatura devanado 2B':[temp_dev2B],
                                 'Temperatura devanado 2C':[temp_dev2C],'Temperatura descanso A':[temp_descA],'Temperatura descanso B':[temp_descB],
                                 'Presion Descarga':[pres_des],'Flujo':[flow],'Velocidad bomba':[velocidad_bomba],'Temperatura bomba':[temp_bomba],
                                 'Aceleracion RMS Horizontal':[acel_hor],'Aceleracion RMS Vertical':[acel_ver],'Aceleracion RMS Axial':[acel_ax],
                                 'Velocidad RMS Horizontal':[vel_hor],'Velocidad RMS Vertical':[vel_ver],'Velocidad RMS Axial':[vel_ax],
                                 'Peak to Peak X':[p2p_x],'Peak to Peak Y':[p2p_y],'Peak to Peak Z':[p2p_z],
                                 'ID Sensor':[id_sensor],'Bateria sensor':[bateria_sensor],'RSSI sensor':[rssi_sensor]})
            par_pp11 = par_pp11.append(pp11,ignore_index=True)
        except:
            print('Conexion no lograda')
            client_par.write_multiple_registers(290,[0])
            pass
        
        ###
        print('PP-012')
        print("Solicitando datos")
        try:
            valores = pedir_datos(PI_string[1])
            pi_av = 1
            time_stamp = json.loads(valores[0].text)['Items'][0]['Value']['Timestamp']
            corr_total = json.loads(valores[0].text)['Items'][0]['Value']['Value']
            hor_total = json.loads(valores[0].text)['Items'][1]['Value']['Value']
            pot_act = json.loads(valores[0].text)['Items'][2]['Value']['Value']
            temp_descA = json.loads(valores[0].text)['Items'][3]['Value']['Value'] # Temperatura descanso ventilacion
            temp_descB = json.loads(valores[0].text)['Items'][4]['Value']['Value'] # Temperatura carga
            temp_dev1A = json.loads(valores[0].text)['Items'][5]['Value']['Value'] # Temperatura devanado 1
            temp_dev1B = json.loads(valores[0].text)['Items'][6]['Value']['Value'] # Temperatura devanado 2
            vdf = json.loads(valores[0].text)['Items'][7]['Value']['Value']
            velocidad_bomba = json.loads(valores[0].text)['Items'][8]['Value']['Value']
            hor_op = json.loads(valores[0].text)['Items'][9]['Value']['Value']
            pump_run = json.loads(valores[0].text)['Items'][10]['Value']['Value']
    
            time_stamp = ((pd.to_datetime(time_stamp,format='%Y-%m-%d %H:%M:%S')).astimezone(new_timezone)).strftime("%Y-%m-%d %H:%M:%S")
            time_stamp_epoch = int((datetime.strptime(time_stamp, "%Y-%m-%d %H:%M:%S")).timestamp())
            tsepoch_bin = bin(time_stamp_epoch)[2:]
            while (len(tsepoch_bin) < 32):
                tsepoch_bin = "0"+ tsepoch_bin
            tsepoch_int_high = int(tsepoch_bin[0:16],2)
            tsepoch_int_low = int(tsepoch_bin[16:],2)
            
            hor_total_bin = bin(np.int64(hor_total))[2:]
            while (len(hor_total_bin) < 48):
                hor_total_bin = "0"+ hor_total_bin
            hor_total_hhigh = int(hor_total_bin[0:16],2)
            hor_total_high = int(hor_total_bin[16:32],2)
            hor_total_low = int(hor_total_bin[32:48],2)

            hor_logica, estado, ef_inst, velocidad_motor, corr_A, corr_B, corr_C  = 1, 0, 0, 0, 0, 0, 0
            temp_dev1C, temp_dev2A, temp_dev2B, temp_dev2C, pres_des, flow, temp_bomba = 0, 0, 0, 0, 0, 0, 0
            acel_hor, acel_ver, acel_ax, vel_hor, vel_ver, vel_ax, p2p_x, p2p_y, p2p_z =  0, 0, 0, 0, 0, 0, 0, 0, 0
            id_sensor, bateria_sensor, rssi_sensor = 0, 0, 0
            
            pump_health = 93
            motor_health = 93
            eff_ind = 91
            operating_hours = np.int16(hor_op)
            ovall_he_pef = np.int16(np.amin([pump_health,motor_health,eff_ind]))            
    
            print("Escribiendo al Modbus")
            client_par.write_multiple_registers(300,[np.int16(pump_run), hor_logica, hor_total_hhigh, hor_total_high, hor_total_low, np.int16(hor_op),
                                                      tsepoch_int_high, tsepoch_int_low, np.int16(estado), np.int16(ef_inst*100)])
            client_par.write_multiple_registers(310,[np.int16(velocidad_motor),np.int16(corr_total), np.int16(corr_A), np.int16(corr_B), np.int16(corr_C),
                                                      np.int16(pot_act),np.int16(temp_dev1A),np.int16(temp_dev1B),np.int16(temp_dev1C),np.int16(temp_dev2A)])
            client_par.write_multiple_registers(320,[np.int16(temp_dev2B),np.int16(temp_dev2C),np.int16(temp_descA),np.int16(temp_descB), np.int16(pres_des),
                                                      np.int16(flow),np.int16(velocidad_bomba),np.int16(temp_bomba),np.int16(acel_hor),np.int16(acel_ver)])
            client_par.write_multiple_registers(330,[np.int16(acel_ax),np.int16(vel_hor),np.int16(vel_ver),np.int16(vel_ax), np.int16(p2p_x), np.int16(p2p_y),
                                                      np.int16(p2p_z),np.int16(id_sensor),np.int16(bateria_sensor),np.int16(rssi_sensor)])
            client_par.write_multiple_registers(340,[pi_av, pump_health, motor_health, eff_ind, operating_hours, ovall_he_pef])
            
            print("Guardando dataframe")
            pp12 = pd.DataFrame({'PI Disponible': [pi_av],'Pump Running':[pump_run],'Horometro Lógica':[hor_logica],'Horometro total':[hor_total],'Horas operando':[hor_op],
                                 'TimeStamp':[time_stamp],'Estado bomba':[estado],'Eficiencia instantanea':[ef_inst],'Velocidad motor':[velocidad_motor],
                                 'Corriente total':[corr_total],'Corriente A':[corr_A],'Corriente B':[corr_B],'Corriente C':[corr_C],
                                 'Potencia Activa':[pot_act],'Temperatura devanado 1A':[temp_dev1A],'Temperatura devanado 1B':[temp_dev1B],
                                 'Temperatura devanado 1C':[temp_dev1C],'Temperatura devanado 2A':[temp_dev2A],'Temperatura devanado 2B':[temp_dev2B],
                                 'Temperatura devanado 2C':[temp_dev2C],'Temperatura descanso A':[temp_descA],'Temperatura descanso B':[temp_descB],
                                 'Presion Descarga':[pres_des],'Flujo':[flow],'Velocidad bomba':[velocidad_bomba],'Temperatura bomba':[temp_bomba],
                                 'Aceleracion RMS Horizontal':[acel_hor],'Aceleracion RMS Vertical':[acel_ver],'Aceleracion RMS Axial':[acel_ax],
                                 'Velocidad RMS Horizontal':[vel_hor],'Velocidad RMS Vertical':[vel_ver],'Velocidad RMS Axial':[vel_ax],
                                 'Peak to Peak X':[p2p_x],'Peak to Peak Y':[p2p_y],'Peak to Peak Z':[p2p_z],
                                 'ID Sensor':[id_sensor],'Bateria sensor':[bateria_sensor],'RSSI sensor':[rssi_sensor]})
            par_pp12 = par_pp12.append(pp12,ignore_index=True)
        except:
            print('Conexion no lograda')
            client_par.write_multiple_registers(340,[0])
            pass
            
        ###
        print('PP-013')
        print("Solicitando datos")
        try:
            valores = pedir_datos(PI_string[2])
            pi_av = 1
            time_stamp = json.loads(valores[0].text)['Items'][0]['Value']['Timestamp']
            corr_total = json.loads(valores[0].text)['Items'][0]['Value']['Value']
            hor_total = json.loads(valores[0].text)['Items'][1]['Value']['Value']
            pot_act = json.loads(valores[0].text)['Items'][2]['Value']['Value']
            temp_descA = json.loads(valores[0].text)['Items'][3]['Value']['Value'] # Temperatura descanso ventilacion
            temp_descB = json.loads(valores[0].text)['Items'][4]['Value']['Value'] # Temperatura carga
            temp_dev1A = json.loads(valores[0].text)['Items'][5]['Value']['Value'] # Temperatura devanado 1
            temp_dev1B = json.loads(valores[0].text)['Items'][6]['Value']['Value'] # Temperatura devanado 2
            vdf = json.loads(valores[0].text)['Items'][7]['Value']['Value']
            velocidad_bomba = json.loads(valores[0].text)['Items'][8]['Value']['Value']
            hor_op = json.loads(valores[0].text)['Items'][9]['Value']['Value']
            pump_run = json.loads(valores[0].text)['Items'][10]['Value']['Value']
    
            time_stamp = ((pd.to_datetime(time_stamp,format='%Y-%m-%d %H:%M:%S')).astimezone(new_timezone)).strftime("%Y-%m-%d %H:%M:%S")
            time_stamp_epoch = int((datetime.strptime(time_stamp, "%Y-%m-%d %H:%M:%S")).timestamp())
            tsepoch_bin = bin(time_stamp_epoch)[2:]
            while (len(tsepoch_bin) < 32):
                tsepoch_bin = "0"+ tsepoch_bin
            tsepoch_int_high = int(tsepoch_bin[0:16],2)
            tsepoch_int_low = int(tsepoch_bin[16:],2)
            
            hor_total_bin = bin(np.int64(hor_total))[2:]
            while (len(hor_total_bin) < 48):
                hor_total_bin = "0"+ hor_total_bin
            hor_total_hhigh = int(hor_total_bin[0:16],2)
            hor_total_high = int(hor_total_bin[16:32],2)
            hor_total_low = int(hor_total_bin[32:48],2)

            hor_logica, estado, ef_inst, velocidad_motor, corr_A, corr_B, corr_C  = 1, 0, 0, 0, 0, 0, 0
            temp_dev1C, temp_dev2A, temp_dev2B, temp_dev2C, pres_des, flow, temp_bomba = 0, 0, 0, 0, 0, 0, 0
            acel_hor, acel_ver, acel_ax, vel_hor, vel_ver, vel_ax, p2p_x, p2p_y, p2p_z =  0, 0, 0, 0, 0, 0, 0, 0, 0
            id_sensor, bateria_sensor, rssi_sensor = 0, 0, 0
            
            pump_health = 93
            motor_health = 93
            eff_ind = 91
            operating_hours = np.int16(hor_op)
            ovall_he_pef = np.int16(np.amin([pump_health,motor_health,eff_ind]))            
    
            print("Escribiendo al Modbus")
            client_par.write_multiple_registers(350,[np.int16(pump_run), hor_logica, hor_total_hhigh, hor_total_high, hor_total_low, np.int16(hor_op),
                                                      tsepoch_int_high, tsepoch_int_low, np.int16(estado), np.int16(ef_inst*100)])
            client_par.write_multiple_registers(360,[np.int16(velocidad_motor),np.int16(corr_total), np.int16(corr_A), np.int16(corr_B), np.int16(corr_C),
                                                      np.int16(pot_act),np.int16(temp_dev1A),np.int16(temp_dev1B),np.int16(temp_dev1C),np.int16(temp_dev2A)])
            client_par.write_multiple_registers(370,[np.int16(temp_dev2B),np.int16(temp_dev2C),np.int16(temp_descA),np.int16(temp_descB), np.int16(pres_des),
                                                      np.int16(flow),np.int16(velocidad_bomba),np.int16(temp_bomba),np.int16(acel_hor),np.int16(acel_ver)])
            client_par.write_multiple_registers(380,[np.int16(acel_ax),np.int16(vel_hor),np.int16(vel_ver),np.int16(vel_ax), np.int16(p2p_x), np.int16(p2p_y),
                                                      np.int16(p2p_z),np.int16(id_sensor),np.int16(bateria_sensor),np.int16(rssi_sensor)])
            client_par.write_multiple_registers(390,[pi_av, pump_health, motor_health, eff_ind, operating_hours, ovall_he_pef])
            
            print("Guardando dataframe")
            pp13 = pd.DataFrame({'PI Disponible': [pi_av],'Pump Running':[pump_run],'Horometro Lógica':[hor_logica],'Horometro total':[hor_total],'Horas operando':[hor_op],
                                 'TimeStamp':[time_stamp],'Estado bomba':[estado],'Eficiencia instantanea':[ef_inst],'Velocidad motor':[velocidad_motor],
                                 'Corriente total':[corr_total],'Corriente A':[corr_A],'Corriente B':[corr_B],'Corriente C':[corr_C],
                                 'Potencia Activa':[pot_act],'Temperatura devanado 1A':[temp_dev1A],'Temperatura devanado 1B':[temp_dev1B],
                                 'Temperatura devanado 1C':[temp_dev1C],'Temperatura devanado 2A':[temp_dev2A],'Temperatura devanado 2B':[temp_dev2B],
                                 'Temperatura devanado 2C':[temp_dev2C],'Temperatura descanso A':[temp_descA],'Temperatura descanso B':[temp_descB],
                                 'Presion Descarga':[pres_des],'Flujo':[flow],'Velocidad bomba':[velocidad_bomba],'Temperatura bomba':[temp_bomba],
                                 'Aceleracion RMS Horizontal':[acel_hor],'Aceleracion RMS Vertical':[acel_ver],'Aceleracion RMS Axial':[acel_ax],
                                 'Velocidad RMS Horizontal':[vel_hor],'Velocidad RMS Vertical':[vel_ver],'Velocidad RMS Axial':[vel_ax],
                                 'Peak to Peak X':[p2p_x],'Peak to Peak Y':[p2p_y],'Peak to Peak Z':[p2p_z],
                                 'ID Sensor':[id_sensor],'Bateria sensor':[bateria_sensor],'RSSI sensor':[rssi_sensor]})
            par_pp13 = par_pp13.append(pp13,ignore_index=True)
        except:
            print('Conexion no lograda')
            client_par.write_multiple_registers(390,[0])
            pass
        
        ###
        print('PP-014')
        print("Solicitando datos")
        try:
            valores = pedir_datos(PI_string[3])
            pi_av = 1
            time_stamp = json.loads(valores[0].text)['Items'][0]['Value']['Timestamp']
            corr_total = json.loads(valores[0].text)['Items'][0]['Value']['Value']
            hor_total = json.loads(valores[0].text)['Items'][1]['Value']['Value']
            pot_act = json.loads(valores[0].text)['Items'][2]['Value']['Value']
            temp_descA = json.loads(valores[0].text)['Items'][3]['Value']['Value'] # Temperatura descanso ventilacion
            temp_descB = json.loads(valores[0].text)['Items'][4]['Value']['Value'] # Temperatura carga
            temp_dev1A = json.loads(valores[0].text)['Items'][5]['Value']['Value'] # Temperatura devanado 1
            temp_dev1B = json.loads(valores[0].text)['Items'][6]['Value']['Value'] # Temperatura devanado 2
            vdf = json.loads(valores[0].text)['Items'][7]['Value']['Value']
            velocidad_bomba = json.loads(valores[0].text)['Items'][8]['Value']['Value']
            hor_op = json.loads(valores[0].text)['Items'][9]['Value']['Value']
            pump_run = json.loads(valores[0].text)['Items'][10]['Value']['Value']
    
            time_stamp = ((pd.to_datetime(time_stamp,format='%Y-%m-%d %H:%M:%S')).astimezone(new_timezone)).strftime("%Y-%m-%d %H:%M:%S")
            time_stamp_epoch = int((datetime.strptime(time_stamp, "%Y-%m-%d %H:%M:%S")).timestamp())
            tsepoch_bin = bin(time_stamp_epoch)[2:]
            while (len(tsepoch_bin) < 32):
                tsepoch_bin = "0"+ tsepoch_bin
            tsepoch_int_high = int(tsepoch_bin[0:16],2)
            tsepoch_int_low = int(tsepoch_bin[16:],2)
            
            hor_total_bin = bin(np.int64(hor_total))[2:]
            while (len(hor_total_bin) < 48):
                hor_total_bin = "0"+ hor_total_bin
            hor_total_hhigh = int(hor_total_bin[0:16],2)
            hor_total_high = int(hor_total_bin[16:32],2)
            hor_total_low = int(hor_total_bin[32:48],2)

            hor_logica, estado, ef_inst, velocidad_motor, corr_A, corr_B, corr_C  = 1, 0, 0, 0, 0, 0, 0
            temp_dev1C, temp_dev2A, temp_dev2B, temp_dev2C, pres_des, flow, temp_bomba = 0, 0, 0, 0, 0, 0, 0
            acel_hor, acel_ver, acel_ax, vel_hor, vel_ver, vel_ax, p2p_x, p2p_y, p2p_z =  0, 0, 0, 0, 0, 0, 0, 0, 0
            id_sensor, bateria_sensor, rssi_sensor = 0, 0, 0
            
            pump_health = 93
            motor_health = 93
            eff_ind = 91
            operating_hours = np.int16(hor_op)
            ovall_he_pef = np.int16(np.amin([pump_health,motor_health,eff_ind]))            
    
            print("Escribiendo al Modbus")
            client_par.write_multiple_registers(400,[np.int16(pump_run), hor_logica, hor_total_hhigh, hor_total_high, hor_total_low, np.int16(hor_op),
                                                      tsepoch_int_high, tsepoch_int_low, np.int16(estado), np.int16(ef_inst*100)])
            client_par.write_multiple_registers(410,[np.int16(velocidad_motor),np.int16(corr_total), np.int16(corr_A), np.int16(corr_B), np.int16(corr_C),
                                                      np.int16(pot_act),np.int16(temp_dev1A),np.int16(temp_dev1B),np.int16(temp_dev1C),np.int16(temp_dev2A)])
            client_par.write_multiple_registers(420,[np.int16(temp_dev2B),np.int16(temp_dev2C),np.int16(temp_descA),np.int16(temp_descB), np.int16(pres_des),
                                                      np.int16(flow),np.int16(velocidad_bomba),np.int16(temp_bomba),np.int16(acel_hor),np.int16(acel_ver)])
            client_par.write_multiple_registers(430,[np.int16(acel_ax),np.int16(vel_hor),np.int16(vel_ver),np.int16(vel_ax), np.int16(p2p_x), np.int16(p2p_y),
                                                      np.int16(p2p_z),np.int16(id_sensor),np.int16(bateria_sensor),np.int16(rssi_sensor)])
            client_par.write_multiple_registers(440,[pi_av, pump_health, motor_health, eff_ind, operating_hours, ovall_he_pef])
            
            print("Guardando dataframe")
            pp14 = pd.DataFrame({'PI Disponible': [pi_av],'Pump Running':[pump_run],'Horometro Lógica':[hor_logica],'Horometro total':[hor_total],'Horas operando':[hor_op],
                                 'TimeStamp':[time_stamp],'Estado bomba':[estado],'Eficiencia instantanea':[ef_inst],'Velocidad motor':[velocidad_motor],
                                 'Corriente total':[corr_total],'Corriente A':[corr_A],'Corriente B':[corr_B],'Corriente C':[corr_C],
                                 'Potencia Activa':[pot_act],'Temperatura devanado 1A':[temp_dev1A],'Temperatura devanado 1B':[temp_dev1B],
                                 'Temperatura devanado 1C':[temp_dev1C],'Temperatura devanado 2A':[temp_dev2A],'Temperatura devanado 2B':[temp_dev2B],
                                 'Temperatura devanado 2C':[temp_dev2C],'Temperatura descanso A':[temp_descA],'Temperatura descanso B':[temp_descB],
                                 'Presion Descarga':[pres_des],'Flujo':[flow],'Velocidad bomba':[velocidad_bomba],'Temperatura bomba':[temp_bomba],
                                 'Aceleracion RMS Horizontal':[acel_hor],'Aceleracion RMS Vertical':[acel_ver],'Aceleracion RMS Axial':[acel_ax],
                                 'Velocidad RMS Horizontal':[vel_hor],'Velocidad RMS Vertical':[vel_ver],'Velocidad RMS Axial':[vel_ax],
                                 'Peak to Peak X':[p2p_x],'Peak to Peak Y':[p2p_y],'Peak to Peak Z':[p2p_z],
                                 'ID Sensor':[id_sensor],'Bateria sensor':[bateria_sensor],'RSSI sensor':[rssi_sensor]})
            par_pp14 = par_pp14.append(pp14,ignore_index=True)
        except:
            print('Conexion no lograda')
            client_par.write_multiple_registers(440,[0])
            pass
            
        ###
        print('PP-015')
        print("Solicitando datos")
        try:
            valores = pedir_datos(PI_string[4])
            time_stamp = json.loads(valores[0].text)['Items'][0]['Value']['Timestamp']
            corr_total = json.loads(valores[0].text)['Items'][0]['Value']['Value']
            hor_total = json.loads(valores[0].text)['Items'][1]['Value']['Value']
            pot_act = json.loads(valores[0].text)['Items'][2]['Value']['Value']
            temp_descA = json.loads(valores[0].text)['Items'][3]['Value']['Value'] # Temperatura descanso ventilacion
            temp_descB = json.loads(valores[0].text)['Items'][4]['Value']['Value'] # Temperatura carga
            temp_dev1A = json.loads(valores[0].text)['Items'][5]['Value']['Value'] # Temperatura devanado 1
            temp_dev1B = json.loads(valores[0].text)['Items'][6]['Value']['Value'] # Temperatura devanado 2
            vdf = json.loads(valores[0].text)['Items'][7]['Value']['Value']
            velocidad_bomba = json.loads(valores[0].text)['Items'][8]['Value']['Value']
            hor_op = json.loads(valores[0].text)['Items'][9]['Value']['Value']
            pump_run = json.loads(valores[0].text)['Items'][10]['Value']['Value']
    
            time_stamp = ((pd.to_datetime(time_stamp,format='%Y-%m-%d %H:%M:%S')).astimezone(new_timezone)).strftime("%Y-%m-%d %H:%M:%S")
            time_stamp_epoch = int((datetime.strptime(time_stamp, "%Y-%m-%d %H:%M:%S")).timestamp())
            tsepoch_bin = bin(time_stamp_epoch)[2:]
            while (len(tsepoch_bin) < 32):
                tsepoch_bin = "0"+ tsepoch_bin
            tsepoch_int_high = int(tsepoch_bin[0:16],2)
            tsepoch_int_low = int(tsepoch_bin[16:],2)
            
            hor_total_bin = bin(np.int64(hor_total))[2:]
            while (len(hor_total_bin) < 48):
                hor_total_bin = "0"+ hor_total_bin
            hor_total_hhigh = int(hor_total_bin[0:16],2)
            hor_total_high = int(hor_total_bin[16:32],2)
            hor_total_low = int(hor_total_bin[32:48],2)

            hor_logica, estado, ef_inst, velocidad_motor, corr_A, corr_B, corr_C  = 1, 0, 0, 0, 0, 0, 0
            temp_dev1C, temp_dev2A, temp_dev2B, temp_dev2C, pres_des, flow, temp_bomba = 0, 0, 0, 0, 0, 0, 0
            acel_hor, acel_ver, acel_ax, vel_hor, vel_ver, vel_ax, p2p_x, p2p_y, p2p_z =  0, 0, 0, 0, 0, 0, 0, 0, 0
            id_sensor, bateria_sensor, rssi_sensor = 0, 0, 0
            
            pump_health = 93
            motor_health = 93
            eff_ind = 91
            operating_hours = np.int16(hor_op)
            ovall_he_pef = np.int16(np.amin([pump_health,motor_health,eff_ind]))            
    
            print("Escribiendo al Modbus")
            client_par.write_multiple_registers(450,[np.int16(pump_run), hor_logica, hor_total_hhigh, hor_total_high, hor_total_low, np.int16(hor_op),
                                                      tsepoch_int_high, tsepoch_int_low, np.int16(estado), np.int16(ef_inst*100)])
            client_par.write_multiple_registers(460,[np.int16(velocidad_motor),np.int16(corr_total), np.int16(corr_A), np.int16(corr_B), np.int16(corr_C),
                                                      np.int16(pot_act),np.int16(temp_dev1A),np.int16(temp_dev1B),np.int16(temp_dev1C),np.int16(temp_dev2A)])
            client_par.write_multiple_registers(470,[np.int16(temp_dev2B),np.int16(temp_dev2C),np.int16(temp_descA),np.int16(temp_descB), np.int16(pres_des),
                                                      np.int16(flow),np.int16(velocidad_bomba),np.int16(temp_bomba),np.int16(acel_hor),np.int16(acel_ver)])
            client_par.write_multiple_registers(480,[np.int16(acel_ax),np.int16(vel_hor),np.int16(vel_ver),np.int16(vel_ax), np.int16(p2p_x), np.int16(p2p_y),
                                                      np.int16(p2p_z),np.int16(id_sensor),np.int16(bateria_sensor),np.int16(rssi_sensor)])
            client_par.write_multiple_registers(490,[pi_av, pump_health, motor_health, eff_ind, operating_hours, ovall_he_pef])
            
            print("Guardando dataframe")
            pp15 = pd.DataFrame({'PI Disponible': [pi_av],'Pump Running':[pump_run],'Horometro Lógica':[hor_logica],'Horometro total':[hor_total],'Horas operando':[hor_op],
                                 'TimeStamp':[time_stamp],'Estado bomba':[estado],'Eficiencia instantanea':[ef_inst],'Velocidad motor':[velocidad_motor],
                                 'Corriente total':[corr_total],'Corriente A':[corr_A],'Corriente B':[corr_B],'Corriente C':[corr_C],
                                 'Potencia Activa':[pot_act],'Temperatura devanado 1A':[temp_dev1A],'Temperatura devanado 1B':[temp_dev1B],
                                 'Temperatura devanado 1C':[temp_dev1C],'Temperatura devanado 2A':[temp_dev2A],'Temperatura devanado 2B':[temp_dev2B],
                                 'Temperatura devanado 2C':[temp_dev2C],'Temperatura descanso A':[temp_descA],'Temperatura descanso B':[temp_descB],
                                 'Presion Descarga':[pres_des],'Flujo':[flow],'Velocidad bomba':[velocidad_bomba],'Temperatura bomba':[temp_bomba],
                                 'Aceleracion RMS Horizontal':[acel_hor],'Aceleracion RMS Vertical':[acel_ver],'Aceleracion RMS Axial':[acel_ax],
                                 'Velocidad RMS Horizontal':[vel_hor],'Velocidad RMS Vertical':[vel_ver],'Velocidad RMS Axial':[vel_ax],
                                 'Peak to Peak X':[p2p_x],'Peak to Peak Y':[p2p_y],'Peak to Peak Z':[p2p_z],
                                 'ID Sensor':[id_sensor],'Bateria sensor':[bateria_sensor],'RSSI sensor':[rssi_sensor]})
            par_pp15 = par_pp15.append(pp15,ignore_index=True)
        except:
            print('Conexion no lograda')
            client_par.write_multiple_registers(490,[0])
            pass
            
        ###
        print('PP-016')
        print("Solicitando datos")
        try:
            valores = pedir_datos(PI_string[5])
            time_stamp = json.loads(valores[0].text)['Items'][0]['Value']['Timestamp']
            corr_total = json.loads(valores[0].text)['Items'][0]['Value']['Value']
            hor_total = json.loads(valores[0].text)['Items'][1]['Value']['Value']
            pot_act = json.loads(valores[0].text)['Items'][2]['Value']['Value']
            temp_descA = json.loads(valores[0].text)['Items'][3]['Value']['Value'] # Temperatura descanso ventilacion
            temp_descB = json.loads(valores[0].text)['Items'][4]['Value']['Value'] # Temperatura carga
            temp_dev1A = json.loads(valores[0].text)['Items'][5]['Value']['Value'] # Temperatura devanado 1
            temp_dev1B = json.loads(valores[0].text)['Items'][6]['Value']['Value'] # Temperatura devanado 2
            vdf = json.loads(valores[0].text)['Items'][7]['Value']['Value']
            velocidad_bomba = json.loads(valores[0].text)['Items'][8]['Value']['Value']
            hor_op = json.loads(valores[0].text)['Items'][9]['Value']['Value']
            pump_run = json.loads(valores[0].text)['Items'][10]['Value']['Value']
    
            time_stamp = ((pd.to_datetime(time_stamp,format='%Y-%m-%d %H:%M:%S')).astimezone(new_timezone)).strftime("%Y-%m-%d %H:%M:%S")
            time_stamp_epoch = int((datetime.strptime(time_stamp, "%Y-%m-%d %H:%M:%S")).timestamp())
            tsepoch_bin = bin(time_stamp_epoch)[2:]
            while (len(tsepoch_bin) < 32):
                tsepoch_bin = "0"+ tsepoch_bin
            tsepoch_int_high = int(tsepoch_bin[0:16],2)
            tsepoch_int_low = int(tsepoch_bin[16:],2)
            
            hor_total_bin = bin(np.int64(hor_total))[2:]
            while (len(hor_total_bin) < 48):
                hor_total_bin = "0"+ hor_total_bin
            hor_total_hhigh = int(hor_total_bin[0:16],2)
            hor_total_high = int(hor_total_bin[16:32],2)
            hor_total_low = int(hor_total_bin[32:48],2)

            hor_logica, estado, ef_inst, velocidad_motor, corr_A, corr_B, corr_C  = 0, 0, 0, 0, 0, 0, 0
            temp_dev1C, temp_dev2A, temp_dev2B, temp_dev2C, pres_des, flow, temp_bomba = 0, 0, 0, 0, 0, 0, 0
            acel_hor, acel_ver, acel_ax, vel_hor, vel_ver, vel_ax, p2p_x, p2p_y, p2p_z =  0, 0, 0, 0, 0, 0, 0, 0, 0
            id_sensor, bateria_sensor, rssi_sensor = 0, 0, 0
            
            pump_health = 93
            motor_health = 93
            eff_ind = 91
            operating_hours = np.int16(hor_op)
            ovall_he_pef = np.int16(np.amin([pump_health,motor_health,eff_ind]))     
    
            print("Escribiendo al Modbus")
            client_par.write_multiple_registers(500,[np.int16(pump_run), hor_logica, hor_total_hhigh, hor_total_high, hor_total_low, np.int16(hor_op),
                                                      tsepoch_int_high, tsepoch_int_low, np.int16(estado), np.int16(ef_inst*100)])
            client_par.write_multiple_registers(510,[np.int16(velocidad_motor),np.int16(corr_total), np.int16(corr_A), np.int16(corr_B), np.int16(corr_C),
                                                      np.int16(pot_act),np.int16(temp_dev1A),np.int16(temp_dev1B),np.int16(temp_dev1C),np.int16(temp_dev2A)])
            client_par.write_multiple_registers(520,[np.int16(temp_dev2B),np.int16(temp_dev2C),np.int16(temp_descA),np.int16(temp_descB), np.int16(pres_des),
                                                      np.int16(flow),np.int16(velocidad_bomba),np.int16(temp_bomba),np.int16(acel_hor),np.int16(acel_ver)])
            client_par.write_multiple_registers(530,[np.int16(acel_ax),np.int16(vel_hor),np.int16(vel_ver),np.int16(vel_ax), np.int16(p2p_x), np.int16(p2p_y),
                                                      np.int16(p2p_z),np.int16(id_sensor),np.int16(bateria_sensor),np.int16(rssi_sensor)])
            client_par.write_multiple_registers(540,[pi_av, pump_health, motor_health, eff_ind, operating_hours, ovall_he_pef])
            
            print("Guardando dataframe")
            pp16 = pd.DataFrame({'PI Disponible': [pi_av],'Pump Running':[pump_run],'Horometro Lógica':[hor_logica],'Horometro total':[hor_total],'Horas operando':[hor_op],
                                 'TimeStamp':[time_stamp],'Estado bomba':[estado],'Eficiencia instantanea':[ef_inst],'Velocidad motor':[velocidad_motor],
                                 'Corriente total':[corr_total],'Corriente A':[corr_A],'Corriente B':[corr_B],'Corriente C':[corr_C],
                                 'Potencia Activa':[pot_act],'Temperatura devanado 1A':[temp_dev1A],'Temperatura devanado 1B':[temp_dev1B],
                                 'Temperatura devanado 1C':[temp_dev1C],'Temperatura devanado 2A':[temp_dev2A],'Temperatura devanado 2B':[temp_dev2B],
                                 'Temperatura devanado 2C':[temp_dev2C],'Temperatura descanso A':[temp_descA],'Temperatura descanso B':[temp_descB],
                                 'Presion Descarga':[pres_des],'Flujo':[flow],'Velocidad bomba':[velocidad_bomba],'Temperatura bomba':[temp_bomba],
                                 'Aceleracion RMS Horizontal':[acel_hor],'Aceleracion RMS Vertical':[acel_ver],'Aceleracion RMS Axial':[acel_ax],
                                 'Velocidad RMS Horizontal':[vel_hor],'Velocidad RMS Vertical':[vel_ver],'Velocidad RMS Axial':[vel_ax],
                                 'Peak to Peak X':[p2p_x],'Peak to Peak Y':[p2p_y],'Peak to Peak Z':[p2p_z],
                                 'ID Sensor':[id_sensor],'Bateria sensor':[bateria_sensor],'RSSI sensor':[rssi_sensor]})
            par_pp16 = par_pp16.append(pp16,ignore_index=True)
        except:
            print('Conexion no lograda')
            client_par.write_multiple_registers(540,[0])
            pass
            
        ###
        print("Guardando excel")
        nombre_archivo = 'PAR_{}-{}-{}.xlsx'.format(now.year,now.month,now.day)
        with pd.ExcelWriter('~/Escritorio/Caserones/datos/PAR/'+nombre_archivo) as writer:  
            par_pp11.to_excel(writer,sheet_name='PP-011')
            par_pp12.to_excel(writer,sheet_name='PP-012')
            par_pp13.to_excel(writer,sheet_name='PP-013')
            par_pp14.to_excel(writer,sheet_name='PP-014')
            par_pp15.to_excel(writer,sheet_name='PP-015')
            par_pp16.to_excel(writer,sheet_name='PP-016')
        sleep(5)
    except:
        now = datetime.now()
        clear_output(wait=True)
        print("Algo fallo, lo intentare de nuevo")
        fecha_error = "Timestamp: {}-{}-{} {}:{}".format(now.year,now.month,now.day,now.hour,now.minute)
        print(fecha_error)
        sleep(5)
        continue

PAR
PP-011
Solicitando datos


Traceback (most recent call last):
  File "/tmp/ipykernel_41750/911109657.py", line 18, in raise_timeout
    raise TimeoutError
TimeoutError
2021-08-18T14:22:19Z


Conexion no lograda
PP-012
Solicitando datos


Traceback (most recent call last):
  File "/tmp/ipykernel_41750/911109657.py", line 18, in raise_timeout
    raise TimeoutError
TimeoutError
2021-08-18T14:22:49Z


Conexion no lograda
PP-013
Solicitando datos
Escribiendo al Modbus
Guardando dataframe
PP-014
Solicitando datos
Escribiendo al Modbus
Guardando dataframe
PP-015
Solicitando datos
